# Data Reading

In [0]:
df = spark.read.format('parquet') \
        .option('inferSchema', True) \
        .load('abfss://bronze@tahirbucket.dfs.core.windows.net/rawdata/')

In [0]:
display(df)

# Data Transformation

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = df.withColumn('model_category', split(col('model_id'), '-')[0])

In [0]:
df.display()

In [0]:
df.withColumn('units_sold', col('units_sold').cast(StringType())).printSchema()

In [0]:
df = df.withColumn('rev_per_unit', col('revenue') / col('units_sold'))
df.display()

# Ad-Hoc

In [0]:
df.groupBy('year', 'branch_name').agg(sum('units_sold').alias('total_units_sold')).sort('year', 'total_units_sold', ascending = [1, 0]).display()

- For visualization on above table remove .display()
- Then put whole query inside display(<query>)

In [0]:
display(df.groupBy('year', 'branch_name').agg(sum('units_sold').alias('total_units_sold')).sort('year', 'total_units_sold', ascending = [1, 0]))

# Data Writing

In [0]:
df.write.format('parquet') \
    .mode('overwrite') \
    .option('path', 'abfss://silver@tahirbucket.dfs.core.windows.net/carsales/') \
    .save()

# Querying silver layer data

In [0]:
%sql
SELECT * FROM PARQUET.`abfss://silver@tahirbucket.dfs.core.windows.net/carsales`